In [2]:
import pandas as pd
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [4]:
SEASONS = list(range(2016,2023))

In [5]:
DATA_DIR = "data"

In [6]:
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [17]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [18]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue
            
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [28]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Refere

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Refere

In [31]:
standings_files = os.listdir(STANDINGS_DIR)

In [43]:
async def scrape_game(standing_file):
    with open(standing_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
        if os.path.exists(save_path): continue

        html = await get_html(url, "#content")
        if not html:
            continue

        with open(save_path, "w+") as f:
            f.write(html)

In [46]:
for f in standings_files: 
    filepath = os.path.join(STANDINGS_DIR, f)
    await scrape_game(filepath)

Knicks vs Suns, January 3, 2020 | Basketball-Reference.com
Pelicans vs Lakers, January 3, 2020 | Basketball-Reference.com
Grizzlies vs Clippers, January 4, 2020 | Basketball-Reference.com
Raptors vs Nets, January 4, 2020 | Basketball-Reference.com
Jazz vs Magic, January 4, 2020 | Basketball-Reference.com
Pacers vs Hawks, January 4, 2020 | Basketball-Reference.com
Thunder vs Cavaliers, January 4, 2020 | Basketball-Reference.com
Celtics vs Bulls, January 4, 2020 | Basketball-Reference.com
Nuggets vs Wizards, January 4, 2020 | Basketball-Reference.com
Hornets vs Mavericks, January 4, 2020 | Basketball-Reference.com
Pistons vs Warriors, January 4, 2020 | Basketball-Reference.com
Spurs vs Bucks, January 4, 2020 | Basketball-Reference.com
Pelicans vs Kings, January 4, 2020 | Basketball-Reference.com
Knicks vs Clippers, January 5, 2020 | Basketball-Reference.com
Trail Blazers vs Heat, January 5, 2020 | Basketball-Reference.com
Timberwolves vs Cavaliers, January 5, 2020 | Basketball-Reference.

Warriors vs Trail Blazers, January 20, 2020 | Basketball-Reference.com
Clippers vs Mavericks, January 21, 2020 | Basketball-Reference.com
Kings vs Pistons, January 22, 2020 | Basketball-Reference.com
Thunder vs Magic, January 22, 2020 | Basketball-Reference.com
76ers vs Raptors, January 22, 2020 | Basketball-Reference.com
Clippers vs Hawks, January 22, 2020 | Basketball-Reference.com
Grizzlies vs Celtics, January 22, 2020 | Basketball-Reference.com
Wizards vs Heat, January 22, 2020 | Basketball-Reference.com
Lakers vs Knicks, January 22, 2020 | Basketball-Reference.com
Timberwolves vs Bulls, January 22, 2020 | Basketball-Reference.com
Nuggets vs Rockets, January 22, 2020 | Basketball-Reference.com
Pacers vs Suns, January 22, 2020 | Basketball-Reference.com
Spurs vs Pelicans, January 22, 2020 | Basketball-Reference.com
Jazz vs Warriors, January 22, 2020 | Basketball-Reference.com
Wizards vs Cavaliers, January 23, 2020 | Basketball-Reference.com
Lakers vs Nets, January 23, 2020 | Basketb

Nets vs Magic, August 11, 2020 | Basketball-Reference.com
Rockets vs Spurs, August 11, 2020 | Basketball-Reference.com
Suns vs 76ers, August 11, 2020 | Basketball-Reference.com
Celtics vs Grizzlies, August 11, 2020 | Basketball-Reference.com
Trail Blazers vs Mavericks, August 11, 2020 | Basketball-Reference.com
Pelicans vs Kings, August 11, 2020 | Basketball-Reference.com
Bucks vs Wizards, August 11, 2020 | Basketball-Reference.com
Pacers vs Rockets, August 12, 2020 | Basketball-Reference.com
Raptors vs 76ers, August 12, 2020 | Basketball-Reference.com
Heat vs Thunder, August 12, 2020 | Basketball-Reference.com
Clippers vs Nuggets, August 12, 2020 | Basketball-Reference.com
Wizards vs Celtics, August 13, 2020 | Basketball-Reference.com
Kings vs Lakers, August 13, 2020 | Basketball-Reference.com
Bucks vs Grizzlies, August 13, 2020 | Basketball-Reference.com
Mavericks vs Suns, August 13, 2020 | Basketball-Reference.com
Spurs vs Jazz, August 13, 2020 | Basketball-Reference.com
Trail Blaze

Kings vs Jazz, April 5, 2019 | Basketball-Reference.com
Pelicans vs Suns, April 5, 2019 | Basketball-Reference.com
Trail Blazers vs Nuggets, April 5, 2019 | Basketball-Reference.com
Cavaliers vs Warriors, April 5, 2019 | Basketball-Reference.com
Lakers vs Clippers, April 5, 2019 | Basketball-Reference.com
Nets vs Bucks, April 6, 2019 | Basketball-Reference.com
76ers vs Bulls, April 6, 2019 | Basketball-Reference.com
Heat vs Raptors, April 7, 2019 | Basketball-Reference.com
Spurs vs Cavaliers, April 7, 2019 | Basketball-Reference.com
Thunder vs Timberwolves, April 7, 2019 | Basketball-Reference.com
Hornets vs Pistons, April 7, 2019 | Basketball-Reference.com
Nets vs Pacers, April 7, 2019 | Basketball-Reference.com
Mavericks vs Grizzlies, April 7, 2019 | Basketball-Reference.com
Suns vs Rockets, April 7, 2019 | Basketball-Reference.com
Hawks vs Bucks, April 7, 2019 | Basketball-Reference.com
Magic vs Celtics, April 7, 2019 | Basketball-Reference.com
Wizards vs Knicks, April 7, 2019 | Bas

2022 NBA Eastern Conference Semifinals Game 2: Bucks vs Celtics, May 3, 2022 | Basketball-Reference.com
2022 NBA Western Conference Semifinals Game 2: Warriors vs Grizzlies, May 3, 2022 | Basketball-Reference.com
2022 NBA Eastern Conference Semifinals Game 2: 76ers vs Heat, May 4, 2022 | Basketball-Reference.com
2022 NBA Western Conference Semifinals Game 2: Mavericks vs Suns, May 4, 2022 | Basketball-Reference.com
2022 NBA Eastern Conference Semifinals Game 3: Heat vs 76ers, May 6, 2022 | Basketball-Reference.com
2022 NBA Western Conference Semifinals Game 3: Suns vs Mavericks, May 6, 2022 | Basketball-Reference.com
2022 NBA Eastern Conference Semifinals Game 3: Celtics vs Bucks, May 7, 2022 | Basketball-Reference.com
2022 NBA Western Conference Semifinals Game 3: Grizzlies vs Warriors, May 7, 2022 | Basketball-Reference.com
2022 NBA Western Conference Semifinals Game 4: Suns vs Mavericks, May 8, 2022 | Basketball-Reference.com
2022 NBA Eastern Conference Semifinals Game 4: Heat vs 76

Error: Page.goto: NS_ERROR_UNKNOWN_HOST
Call log:
navigating to "https://www.basketball-reference.com/boxscores/201902020CLE.html", waiting until "load"
